In [6]:
import time
import numpy as np
from scipy import spatial as spatial
import pandas as pd
import random
import math

In [17]:
# read dat file 
datContent = [i.strip().split() for i in open("../nsta24_sac.dat").readlines()]
# write it as a dataframe
df=pd.DataFrame(datContent,
                columns=["station name","lon","lat","?","?","?","agency","CSMT","?","?","?","?","?","?","date","?","?","?","?","?","?","?","?","?","?","?","?","?","?","?","?"]
               )
# filter CSMT
new_df=df[df["CSMT"]=="CSMT"]
new_df.to_csv('../existing_station.dat', sep=' ', header=None,index=None)
exist_lon=pd.to_numeric(new_df["lon"])
exist_lat=pd.to_numeric(new_df["lat"])
exist_lon_lat=np.array([exist_lon,exist_lat]).transpose()

In [19]:
#define the area of interest
lon_start = 120
lon_end = 122
lat_start= 21.8
lat_end= 25.5

step_count=0.025
step_lon= (lon_end-lon_start)/step_count
step_lat= (lat_end-lat_start)/step_count

#set of grid
#initialize grid storage
lon_storage=np.array([])
lat_storage=np.array([])

#compute individual points
for i in range(int(step_lon+2)):
    for j in range(int(step_lat+2)):
        lat_storage=np.append(lat_storage,lat_start+j*step_count)
        lon_storage=np.append(lon_storage,lon_start+i*step_count)

#add the two arrays together
grid_point_base= np.concatenate(([lon_storage],[lat_storage]))
#len(grid_point_base[0])
grid_point_base

array([[120.   , 120.   , 120.   , ..., 122.025, 122.025, 122.025],
       [ 21.8  ,  21.825,  21.85 , ...,  25.45 ,  25.475,  25.5  ]])

In [19]:
#merge exisiting stations in each grid
s=pd.DataFrame(exist_lon_lat,columns=["lon","lat"])
s1=pd.cut(s["lon"], np.arange(lon_start, lon_end, step_count))
s2=pd.cut(s["lat"], np.arange(lat_start, lat_end, step_count))
result=s.groupby([s1,s2]).mean()

lon_lat_nan=np.array([result["lon"],result["lat"]]).transpose()

#remove NaN
lon_lat_nan = lon_lat_nan[np.logical_not(np.isnan(lon_lat_nan))]
lon=np.array([])
lat=np.array([])

for i in range(len(lon_lat_nan)):
    if i%2==0:
        lon=np.append(lon,lon_lat_nan[i])
    else:
        lat=np.append(lat,lon_lat_nan[i])
lon_lat= np.concatenate(([lon],[lat])).transpose()


In [20]:
#define converter: lon lat to x y coords
def converter(lon2,lat2):
    lon1=lon_start
    lat1=lat_start
    dx = (lon2-lon1)*40000*math.cos((lat1+lat2)*math.pi/360)/360
    dy = (lat2-lat1)*40000/360
    return(np.array([dx,dy]))


In [21]:
#define the inputs
points=grid_point_base
combined_x_y_arrays=lon_lat
#transpose and list points
points_list = list(points.transpose())


In [22]:
#define kd tree function
def do_kdtree(combined_x_y_arrays,points):
    mytree = spatial.cKDTree(combined_x_y_arrays)
    dist, indexes = mytree.query(points)
    return dist, indexes


In [23]:
#initialize storages
storage=np.array([])
average_distance_storage=np.array([])

#run kd tree function
start = time.time()
iteration=1
for i in range(len(points_list)):
#for i in range(1000):
    storage=np.array([])
    combined_x_y_arrays=lon_lat
    points_list = list(points.transpose())
    for k in range(iteration):
        #do_kd_tree
        kd_result = do_kdtree(combined_x_y_arrays,points_list)
        result_compiler=pd.DataFrame(kd_result, index=["dist", "indexes"]).transpose()
        #store the nearest point
        #storage=np.append(storage, result_compiler["dist"][i])
        #convert lon/lat to the x/y
        lon_end,lat_end=combined_x_y_arrays[int(result_compiler["indexes"][i])]
        individual_x,individual_y=points_list[i]
        #calculate the distance
        dist=np.linalg.norm(converter(lon_end,lat_end)-converter(individual_x,individual_y))
        storage=np.append(storage, dist)
        #remove the nearest point
        combined_x_y_arrays=np.delete(combined_x_y_arrays, int(result_compiler["indexes"][i]),0)
    average_distance=np.array(sum(storage)/iteration)
    average_distance_storage=np.append(average_distance_storage,average_distance)
end = time.time()
print('Completed in: ',end-start)

Completed in:  10866.073817968369


In [24]:
#put the relevant data into dataframe
grid_value=pd.DataFrame(data=average_distance_storage.reshape(1,-1), index=["average_dist"])
cords=pd.DataFrame(data=points,index=["lon","lat"])
grid_value_and_cords=cords.append(grid_value)
grid_value_and_cords=grid_value_and_cords.transpose()

In [25]:
pd.set_option('display.max_rows', grid_value_and_cords.shape[0]+1)
grid_value_and_cords

,lon,lat,average_dist
0,120.000,21.800,83.822217
1,120.000,21.825,85.206129
2,120.000,21.850,83.474381
3,120.000,21.875,81.056356
4,120.000,21.900,78.662096
5,120.000,21.925,76.293840
6,120.000,21.950,73.954086
7,120.000,21.975,71.645626
8,120.000,22.000,69.371584
9,120.000,22.025,67.135459


In [26]:
grid_value_and_cords.to_csv('../heatmap_real.txt', sep=' ', header=None,index=None)

In [28]:
list(points.transpose())

[array([120. ,  21.8]),
 array([120.   ,  21.825]),
 array([120.  ,  21.85]),
 array([120.   ,  21.875]),
 array([120. ,  21.9]),
 array([120.   ,  21.925]),
 array([120.  ,  21.95]),
 array([120.   ,  21.975]),
 array([120.,  22.]),
 array([120.   ,  22.025]),
 array([120.  ,  22.05]),
 array([120.   ,  22.075]),
 array([120. ,  22.1]),
 array([120.   ,  22.125]),
 array([120.  ,  22.15]),
 array([120.   ,  22.175]),
 array([120. ,  22.2]),
 array([120.   ,  22.225]),
 array([120.  ,  22.25]),
 array([120.   ,  22.275]),
 array([120. ,  22.3]),
 array([120.   ,  22.325]),
 array([120.  ,  22.35]),
 array([120.   ,  22.375]),
 array([120. ,  22.4]),
 array([120.   ,  22.425]),
 array([120.  ,  22.45]),
 array([120.   ,  22.475]),
 array([120. ,  22.5]),
 array([120.   ,  22.525]),
 array([120.  ,  22.55]),
 array([120.   ,  22.575]),
 array([120. ,  22.6]),
 array([120.   ,  22.625]),
 array([120.  ,  22.65]),
 array([120.   ,  22.675]),
 array([120. ,  22.7]),
 array([120.   ,  22.725

In [29]:
lon_lat

array([[120.1168 ,  23.0768 ],
       [120.1392 ,  23.2621 ],
       [120.1661 ,  23.0006 ],
       [120.1621 ,  23.2191 ],
       [120.1597 ,  23.3304 ],
       [120.1535 ,  23.4607 ],
       [120.1906 ,  22.90785],
       [120.1897 ,  22.9732 ],
       [120.1988 ,  23.1226 ],
       [120.1752 ,  23.2377 ],
       [120.1795 ,  23.5996 ],
       [120.193  ,  23.7025 ],
       [120.2242 ,  22.8143 ],
       [120.2172 ,  22.9188 ],
       [120.2115 ,  23.0089 ],
       [120.2482 ,  22.7834 ],
       [120.2407 ,  22.9813 ],
       [120.2458 ,  23.1234 ],
       [120.2441 ,  23.2926 ],
       [120.2396 ,  23.4835 ],
       [120.2491 ,  23.7046 ],
       [120.272  ,  22.6084 ],
       [120.2643 ,  22.7601 ],
       [120.2643 ,  22.8675 ],
       [120.2599 ,  23.1817 ],
       [120.2543 ,  23.751  ],
       [120.2874 ,  22.6229 ],
       [120.2926 ,  22.6905 ],
       [120.2889 ,  22.9648 ],
       [120.2957 ,  23.0791 ],
       [120.2769 ,  23.2137 ],
       [120.2992 ,  23.2977 ],
       [